In [1]:
import pandas as pd

In [2]:
import os
print("Carpeta actual:", os.getcwd())
print("Archivos y carpetas:", os.listdir())

Carpeta actual: C:\Users\nerea\Documents\Repositorio\python
Archivos y carpetas: ['.git', 'currency_info.json', 'food_crisis', 'random_colors', 'social_media']


In [3]:
dtype_dict = {
    "countryiso3": "str",
    "date": "str",  
    "admin1": "str",
    "admin2": "str",
    "market": "str",
    "market_id": "str", 
    "latitude": "float64",
    "longitude": "float64",
    "category": "str",
    "commodity": "str",
    "commodity_id": "str", 
    "unit": "str",
    "priceflag": "str",
    "pricetype": "str",
    "currency": "str",
    "price": "float64",
    "usdprice": "float64"
}

# Cargar el dataset
df = pd.read_csv("food_crisis/data/wfp_food_prices_global.csv", dtype=dtype_dict, skiprows=1) 

In [4]:
print("\Variables en el conjunto de datos: ")
print(df.columns)

\Variables en el conjunto de datos: 
Index(['#country+code', '#date', '#adm1+name', '#adm2+name',
       '#loc+market+name', '#loc+market+code', '#geo+lat', '#geo+lon',
       '#item+type', '#item+name', '#item+code', '#item+unit',
       '#item+price+flag', '#item+price+type', '#currency+code', '#value',
       '#value+usd'],
      dtype='object')


### Renombramiento de variables
A continuación, se van a renombrar las columnas para mayor claridad.

In [5]:
# Renombramiendo de columnas para mayor claridad
df.rename(columns={
    '#country+code': 'countryiso3',
    '#date': 'date',
    '#adm1+name': 'admin1',
    '#adm2+name': 'admin2',
    '#loc+market+name': 'market',
    '#loc+market+code': 'market_id',
    '#geo+lat': 'latitude',
    '#geo+lon': 'longitude',
    '#item+type': 'category',
    '#item+name': 'commodity',
    '#item+code': 'commodity_id',
    '#item+unit': 'unit',
    '#item+price+flag': 'priceflag',
    '#item+price+type': 'pricetype',
    '#currency+code': 'currency',
    '#value': 'price',
    '#value+usd': 'usdprice'
}, inplace=True)

In [6]:
print("\Variables en el conjunto de datos: ")
print(df.columns)

\Variables en el conjunto de datos: 
Index(['countryiso3', 'date', 'admin1', 'admin2', 'market', 'market_id',
       'latitude', 'longitude', 'category', 'commodity', 'commodity_id',
       'unit', 'priceflag', 'pricetype', 'currency', 'price', 'usdprice'],
      dtype='object')


El nuevo listado se corresponde con: 

* countryiso3: código del país en formato ISO de 3 letras
* date: fecha de registro de los datos
* admin1: nombre de la primera división administrativa de cada país
* admin2: nombre de la segunda división administrativa de cada país
* market: mercado en el que se registra ese precio
* market_id: identificador único para el mercado
* latitude: coordenada latitud del mercado
* longitude: coordenada longitud del mercado
* category: categoria del producto
* commodity: producto 
* commodity_id: identificador del producto
* unit: unidad de medida para el precio
* priceflag: tipo de precio
* pricetype: distribución y venta de tipo al por menor (retail) o al por mayor (wholesale) 
* currency: moneda en la que se registra el precio (ISO 4217)
* price: precio local en la moneda local
* usdprice: precio local equivalente en USD

### Pycountry para cambiar códigos ISO de países por sus nombre
Ahora se va a utilizar la librería pycountry para acceder al listado de países con sus códigos ISO y nombres en inglés para poder importarlo desde un json y usarlo en lugar de los códigos en formato ISO 3

In [7]:
from food_crisis.utils.country_mapping import country_map

df['countryiso3'] = df['countryiso3'].replace(country_map)


### Forex-python para cambiar códigos ISO de monedas por su nombre y símbolo

In [9]:
from food_crisis.utils.currency_mapping import map_currency

currency_ISO = df['currency'].unique()
# print(currency_ISO)

currency_mapped = map_currency(currency_ISO)
print((currency_mapped))

df['currency'] = df['currency'].replace(currency_mapped)

{'AFN': 'Afghan afghani (Afs)', 'AOA': 'Angolan kwanza (Kz)', 'ARS': 'Argentine peso ($)', 'AMD': 'Armenian dram (AMD)', 'BIF': 'Burundi franc (FBu)', 'XOF': 'West African CFA franc (CFA)', 'BDT': 'Bangladeshi taka (৳)', 'BYR': 'Belarusian ruble (Br)', 'BOB': 'Bolivian boliviano (Bs.)', 'BTN': 'Bhutanese ngultrum (Nu.)', 'XAF': 'Central African CFA franc (CFA)', 'CNY': 'Chinese/Yuan renminbi (¥)', 'CDF': 'Congolese franc (F)', 'COP': 'Colombian peso (Col$)', 'CVE': 'Cape Verdean escudo (Esc)', 'DJF': 'Djiboutian franc (Fdj)', 'DOP': 'Dominican peso (RD$)', 'DZD': 'Algerian dinar (د.ج)', 'USD': 'United States dollar ($)', 'EGP': 'Egyptian pound (£)', 'ERN': 'Eritrean nakfa (Nfa)', 'ETB': 'Ethiopian birr (Br)', 'FJD': 'Fijian dollar (FJ$)', 'GEL': 'Georgian lari (GEL)', 'GHS': 'Ghanaian cedi (GH₵)', 'GNF': 'Guinean franc (FG)', 'GMD': 'Gambian dalasi (D)', 'GTQ': 'Guatemalan quetzal (Q)', 'HNL': 'Honduran lempira (L)', 'HTG': 'Haitian gourde (G)', 'IDR': 'Indonesian rupiah (Rp)', 'INR': 

## Limpieza ETL de datos

En este paso se eliminan los valores nulos y duplicados, se aplica la normalización de nombres a las columnas, y se convierten los tipos de datos. También se filtran valores inconsistentes como precios negativos, y se les da un nombre más claro a las columnas.
Finalmente, se guardan los datos limpios en un nuevo archivo CSV.

In [10]:
# Eliminar valores nulos
df.dropna(inplace=True) 

# Normalización de nombre de columnas 
df.columns = df.columns.str.lower().str.replace(" ", "_").str.replace("(", "").str.replace(")", "")

# Conversión de tipos  de datos
df['date'] = pd.to_datetime(df['date'])
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['usdprice'] = pd.to_numeric(df['usdprice'], errors='coerce')
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')



C:\Users\nerea\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


### Creación de variables nuevas útiles (Feature Engineering)

Se pueden derivar información relevante como el mes y el año a partir de "date", y la región codificada a partir de admin1/admin2.

In [11]:
# Extrae el año, mes, día y día de la semana de la fecha 
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday

df['region'] = df['admin1'] + ' - ' + df['admin2']

In [12]:
# Eliminación de duplicados
df.drop_duplicates(inplace=True)

# Filtrado de valores inconsistentes 
df = df[df['price'] > 0]
df = df[df['usdprice'] > 0]

In [13]:
# Obtiene una muestra para reducir tiempos 
reduced_df = df.sample(frac=0.01, random_state=42)

In [14]:
reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8266 entries, 115412 to 301164
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   countryiso3   8266 non-null   object        
 1   date          8266 non-null   datetime64[ns]
 2   admin1        8266 non-null   object        
 3   admin2        8266 non-null   object        
 4   market        8266 non-null   object        
 5   market_id     8266 non-null   int64         
 6   latitude      8266 non-null   float64       
 7   longitude     8266 non-null   float64       
 8   category      8266 non-null   object        
 9   commodity     8266 non-null   object        
 10  commodity_id  8266 non-null   int64         
 11  unit          8266 non-null   object        
 12  priceflag     8266 non-null   object        
 13  pricetype     8266 non-null   object        
 14  currency      8266 non-null   object        
 15  price         8266 non-null   f

In [15]:
# Guardado de datos limmpios
reduced_df.to_csv("food_crisis/data/clean_food_prices.csv", index=False)